In [3]:
import os

import numpy as np
import pandas as pd
from utils.file_utilities import s3_download

### Import snp500 constituents


In [2]:
path = "Index/snp500constituents2023.csv"
file_path = s3_download(path)
index_constituents = pd.read_csv(file_path)
index_constituents.head()

,PERMNO,Company Name,Ticker,SP500 Start,SP500 End
0,10104,ORACLE CORP,ORCL,"Aug. 3, 1989","Dec. 29, 2023"
1,10107,MICROSOFT CORP,MSFT,"June 7, 1994","Dec. 29, 2023"
2,10138,T ROWE PRICE GROUP INC,TROW,"Oct. 13, 1999","Dec. 29, 2023"
3,10145,HONEYWELL INTERNATIONAL INC,HON,"Dec. 31, 1925","Dec. 29, 2023"
4,10516,ARCHER DANIELS MIDLAND CO,ADM,"July 30, 1981","Dec. 29, 2023"


In [6]:
index_constituents["SP500 Start"] = pd.to_datetime(index_constituents["SP500 Start"], format="mixed")
index_constituents["SP500 End"] = pd.to_datetime(index_constituents["SP500 End"], format="mixed")

In [9]:
index_start = pd.Timestamp("2023-01-01")
index_end = pd.Timestamp("2023-12-29")

index_ids = index_constituents[
    (index_constituents["SP500 Start"] < index_start) & (index_constituents["SP500 End"] == index_end)
]
index_ids

,PERMNO,Company Name,Ticker,SP500 Start,SP500 End
0,10104,ORACLE CORP,ORCL,1989-08-03,2023-12-29
1,10107,MICROSOFT CORP,MSFT,1994-06-07,2023-12-29
2,10138,T ROWE PRICE GROUP INC,TROW,1999-10-13,2023-12-29
3,10145,HONEYWELL INTERNATIONAL INC,HON,1925-12-31,2023-12-29
4,10516,ARCHER DANIELS MIDLAND CO,ADM,1981-07-30,2023-12-29
...,...,...,...,...,...
498,93096,DOLLAR GENERAL CORP NEW,DG,2012-12-03,2023-12-29
499,93132,FORTINET INC,FTNT,2018-10-11,2023-12-29
500,93246,GENERAC HOLDINGS INC,GNRC,2021-03-22,2023-12-29
501,93429,C B O E GLOBAL MARKETS INC,CBOE,2017-03-01,2023-12-29


In [13]:
index_ids["PERMNO"].to_string("../../data/sp500_ids.txt", index=False)

### Import snp500 constituents


In [15]:
path = "Index/DowJonesAndNasdaq100.csv"
file_path = s3_download(path)
index_constituents = pd.read_csv(file_path)
index_constituents.head()

,gvkey,iid,gvkeyx,from,thru,conm,indextype,tic,spii,spmi,indexcat,co_conm,co_tic,co_cusip,co_cik,co_sic,co_naics
0,5047,1,5,11/07/1907,06/25/2018,Dow Jones Industrials-30 Stk,NONE,I0005,NaN,NaN,DOW,GE AEROSPACE,GE,369604301,40545.0,9997,999977
1,4503,1,5,10/01/1928,08/28/2020,Dow Jones Industrials-30 Stk,NONE,I0005,NaN,NaN,DOW,EXXON MOBIL CORP,XOM,30231G102,34088.0,2911,324110
2,3144,1,5,05/26/1932,NaN,Dow Jones Industrials-30 Stk,NONE,I0005,NaN,NaN,DOW,COCA-COLA CO,KO,191216100,21344.0,2086,312111
3,6066,1,5,05/26/1932,NaN,Dow Jones Industrials-30 Stk,NONE,I0005,NaN,NaN,DOW,INTL BUSINESS MACHINES CORP,IBM,459200101,51143.0,7370,518210
4,8762,1,5,05/26/1932,NaN,Dow Jones Industrials-30 Stk,NONE,I0005,NaN,NaN,DOW,PROCTER & GAMBLE CO,PG,742718109,80424.0,2840,325611


In [20]:
len(index_constituents["co_cusip"].unique())

294

In [21]:
index_constituents["co_cusip"].to_string("../../data/djn100_tickers.txt", index=False)

### Join data

Have to separately upload both txt files into WRDS, given different identifiers:

https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/stock-header-info/

Then we got a joined unique set of securities and uploaded into S3.

https://us-east-1.console.aws.amazon.com/s3/object/capstone-bucket-4-friends?region=us-east-1&bucketType=general&prefix=Index/security_master.csv

594 stocks were selected in the end.


## filter the crsp_2023_clean


In [ ]:
file_path = s3_download("CRSP/crsp_2023_clean.csv")
df = pd.read_csv(file_path)
df["date"] = pd.to_datetime(df["date"])

In [ ]:
df.dtypes

In [ ]:
df_sample = df[df["PERMNO"].isin(index_ids["PERMNO"])]
df_pivot = df_sample.pivot(index="date", columns="PERMNO", values="RET")
df_pivot.head()

In [ ]:
date_cutoff = pd.Timestamp("2023-10-01")

In [ ]:
train = df_pivot[:date_cutoff]
train_span = len(train)
train_span

In [ ]:
test = df_pivot[date_cutoff:]
test_span = len(test)
test_span

In [ ]:
train.index

Reference: https://github.com/zhuodannychen/Portfolio-Optimization
